# Data Science Prompt Engineer - Fine-Tuning with Unsloth on Colab

This notebook fine-tunes a 4-bit quantized LLM using Unsloth on Google Colab to generate Python code for data analysis prompts. It uses the `sahil2801/CodeAlpaca-20k` dataset filtered for Python examples.

## Install Dependencies

In [ ]:
!pip install -U "triton==2.0.0"

In [2]:
!pip install unsloth
!pip install --no-deps xformers trl peft accelerate bitsandbytes datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 3.8 MB/s eta 0:00:00
  Using cached torch-2.8.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (30 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.8.93-py3-none-manylinux2010_x86_64.manylinux_2_12_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cuda_runtime_cu12-12.8.90-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cuda_cupti_cu12-12.8.90-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cudnn_cu12-9.10.2.21-py3-none-manylinux_2_27_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_cublas_cu12-12.8.4.1-py3-none-manylinux_2_27_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cufft_cu12-11.3.3.83-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_curand_cu12-10.3.9.90-py3-none-manylinux_2_27_x86_64.whl.metadata (1.7 kB)
  Using 

## Login to Hugging Face

In [3]:
from huggingface_hub import login
from google.colab import userdata

hf_token = userdata.get("HuggingFace")  # Add your HF token in Colab Secrets
login(token=hf_token)

## Load Model with Unsloth

In [ ]:
from unsloth import FastLanguageModel

max_seq_length = 2048
load_in_4bit = True

dtype = None  # Auto-detect dtype

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/mistral-7b-v0.3-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

## Load and Format Dataset

In [5]:
from datasets import load_dataset

dataset = load_dataset("sahil2801/CodeAlpaca-20k", split="train")

def filter_python(example):
    return "python" in example["output"].lower()

dataset = dataset.filter(filter_python)

README.md:   0%|          | 0.00/147 [00:00<?, ?B/s]

code_alpaca_20k.json: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/20022 [00:00<?, ? examples/s]

Filter:   0%|          | 0/20022 [00:00<?, ? examples/s]

## Define Prompt Format

In [6]:
prompt_style = """You are an Experienced Data Science Assistant. Given an instruction, write the appropriate Python code.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs = examples.get("input", [""] * len(instructions))
    outputs = examples["output"]
    texts = [prompt_style.format(inst, inp, out) + EOS_TOKEN for inst, inp, out in zip(instructions, inputs, outputs)]
    return {"text": texts}

dataset = dataset.map(formatting_prompts_func, batched=True)

Map:   0%|          | 0/374 [00:00<?, ? examples/s]

## Add PEFT and Trainer

In [9]:
from trl import SFTTrainer
from transformers import TrainingArguments
import torch

model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = False,
    random_state = 42,
    max_seq_length = max_seq_length,
)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 30,  # Increase for better results
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 42,
        output_dir = "outputs",
    ),
)

Unsloth: Already have LoRA adapters! We shall skip this step.


Unsloth: Tokenizing ["text"]:   0%|          | 0/374 [00:00<?, ? examples/s]

## Train the Model

In [10]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 374 | Num Epochs = 1 | Total steps = 30
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 4 x 1) = 4
 "-____-"     Trainable parameters = 41,943,040 of 7,289,966,592 (0.58% trained)


Step,Training Loss
1,0.534800
2,0.575200
3,0.623700
4,0.642600
5,0.806200
6,0.707200
7,0.550600
8,0.599500
9,0.513900
10,0.650100


## Save and Push to Hugging Face

In [11]:
new_model_name = "ZyrexAN/DataScience-PromptEngineer"

model.save_pretrained(new_model_name)
tokenizer.save_pretrained(new_model_name)

model.push_to_hub(new_model_name, use_auth_token=True)
tokenizer.push_to_hub(new_model_name, use_auth_token=True)

README.md:   0%|          | 0.00/588 [00:00<?, ?B/s]

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...ptpedxkmg/adapter_model.safetensors:   0%|          |  559kB /  168MB            

Saved model to https://huggingface.co/ZyrexAN/DataScience-PromptEngineer


Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  /tmp/tmpz7l6mno5/tokenizer.model      : 100%|##########|  587kB /  587kB            

## Fin!
The model is ready now!